#链接谷歌云盘和Colab

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131289 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

#查看当前路径

In [2]:
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive
import os
os.chdir("drive")
!ls

aclImdb_v1.tar.gz     residual_plot.png      Titanic
data_description.txt  Sales		     torchtest.ipynb
drive		      sample_submission.csv  train.csv
GuodongDrive	      Sense1.ipynb	     Untitled0.ipynb
House		      Sense2.ipynb（副本）   XGboost_practice1.ipynb
Houseprice.ipynb      Sense.ipynb	     XGboost_practice2.ipynb
huawei		      sensePY.ipynb	     剑指offer
labeledTrainData.tsv  submission.csv	     数组和链表.desktop
neg		      test.csv		     糖尿病数据.txt
practice	      testData.tsv


In [3]:
os.chdir("practice")
os.chdir("ch7")
!ls

air_data.csv  zscoredata.xls   航空公司客户价值分析.ipynb
ch_en.xls     zscoreddata.xls


#开始

programmer_1-->关于原始数据的一些特征描述并
保存为新表，使用describe函数

programmer_2-->对原始数据进行清理，对其中某
些数据做运算，并进行保存

programmer_3-->标准化数据并进行保存

programmer_4-->使用KMeans对数据进行聚类分析

In [0]:
import pandas as pd
from sklearn.cluster import KMeans

In [8]:
data = pd.read_csv('air_data.csv',encoding = 'utf-8')
# 包括对数据的基本描述，percentiles参数是指定计算多少的分位数表（如1/4分位数、中位数等）；
#T是转置，转置后更方便查阅
explore = data.describe(percentiles = [],include = 'all').T
#describe函数自动计算非空值数，需手动计算空值数
explore['null'] = len(data) - explore['count']
explore = explore[['null','max','min']]
explore.columns = [u'空值数',u'最大值',u'最小值']
#explore = explore.drop('MEMBER_NO')
explore.head()

,空值数,最大值,最小值
MEMBER_NO,0,62988,1
FFP_DATE,0,NaN,NaN
FIRST_FLIGHT_DATE,0,NaN,NaN
GENDER,3,NaN,NaN
FFP_TIER,0,6,4


In [10]:
# 使用乘法运算非空数值的数据，因为numpy不支持*运算，在这里换做&运算
data = data[data['SUM_YR_1'].notnull() & data['SUM_YR_2'].notnull()]
# 只保留票价非零的，或者平均折扣率与总飞行公里数同时为0的记录。
index1 = data['SUM_YR_1'] != 0
index2 = data['SUM_YR_2'] != 0
index3 = (data['SEG_KM_SUM'] == 0) & (data['avg_discount'] == 0) # 该规则是“与”
data = data[index1 | index2 | index3]  # 该规则是“或”
data.head()

,MEMBER_NO,FFP_DATE,FIRST_FLIGHT_DATE,GENDER,FFP_TIER,WORK_CITY,WORK_PROVINCE,WORK_COUNTRY,AGE,LOAD_TIME,FLIGHT_COUNT,BP_SUM,EP_SUM_YR_1,EP_SUM_YR_2,SUM_YR_1,SUM_YR_2,SEG_KM_SUM,WEIGHTED_SEG_KM,LAST_FLIGHT_DATE,AVG_FLIGHT_COUNT,AVG_BP_SUM,BEGIN_TO_FIRST,LAST_TO_END,AVG_INTERVAL,MAX_INTERVAL,ADD_POINTS_SUM_YR_1,ADD_POINTS_SUM_YR_2,EXCHANGE_COUNT,avg_discount,P1Y_Flight_Count,L1Y_Flight_Count,P1Y_BP_SUM,L1Y_BP_SUM,EP_SUM,ADD_Point_SUM,Eli_Add_Point_Sum,L1Y_ELi_Add_Points,Points_Sum,L1Y_Points_Sum,Ration_L1Y_Flight_Count,Ration_P1Y_Flight_Count,Ration_P1Y_BPS,Ration_L1Y_BPS,Point_NotFlight
0,54993,2006/11/02,2008/12/24,男,6,.,北京,CN,31.0,2014/03/31,210,505308,0,74460,239560.0,234188.0,580717,558440.14,2014/03/31,26.250,63163.500,2,1,3.483254,18,3352,36640,34,0.961639,103,107,246197,259111,74460,39992,114452,111100,619760,370211,0.509524,0.490476,0.487221,0.512777,50
1,28065,2007/02/19,2007/08/03,男,6,NaN,北京,CN,42.0,2014/03/31,140,362480,0,41288,171483.0,167434.0,293678,367777.20,2014/03/25,17.500,45310.000,2,7,5.194245,17,0,12000,29,1.252314,68,72,177358,185122,41288,12000,53288,53288,415768,238410,0.514286,0.485714,0.489289,0.510708,33
2,55106,2007/02/01,2007/08/30,男,6,.,北京,CN,40.0,2014/03/31,135,351159,0,39711,163618.0,164982.0,283712,355966.50,2014/03/21,16.875,43894.875,10,11,5.298507,18,3491,12000,20,1.254676,65,70,169072,182087,39711,15491,55202,51711,406361,233798,0.518519,0.481481,0.481467,0.518530,26
3,21189,2008/08/22,2008/08/23,男,5,Los Angeles,CA,US,64.0,2014/03/31,23,337314,0,34890,116350.0,125500.0,281336,306900.88,2013/12/26,2.875,42164.250,21,97,27.863636,73,0,0,11,1.090870,13,10,186104,151210,34890,0,34890,34890,372204,186100,0.434783,0.565217,0.551722,0.448275,12
4,39546,2009/04/10,2009/04/15,男,6,贵阳,贵州,CN,48.0,2014/03/31,152,273844,0,42265,124560.0,130702.0,309928,300834.06,2014/03/27,19.000,34230.500,3,5,4.788079,47,0,22704,27,0.970658,71,81,128448,145396,42265,22704,64969,64969,338813,210365,0.532895,0.467105,0.469054,0.530943,39


In [18]:
data = pd.read_excel('zscoredata.xls')
data = (data- data.mean(axis = 0))/(data.std(axis = 0))
data.columns = ['Z' + i for i in data.columns]
data.head()

,ZL,ZR,ZF,ZM,ZC
0,1.689882,0.140299,-0.635788,0.068794,-0.337186
1,1.689882,-0.322442,0.852453,0.843848,-0.553613
2,1.681743,-0.487707,-0.210576,0.158569,-1.094680
3,1.534185,-0.785184,0.002030,0.273091,-1.148787
4,0.890167,-0.426559,-0.635788,-0.685170,1.231909


In [0]:
data.to_excel('processed_zscoredata.xls', index=False)


In [0]:
data = pd.read_excel('processed_zscoredata.xls')

In [24]:
  k = 5
  kmodel = KMeans(n_clusters=k, n_jobs=3)
  kmodel.fit(data)

  print(kmodel.cluster_centers_)  # 查看聚类中心
  print(kmodel.labels_)  # 查看各样本对应的类别

[[ 0.48347647 -0.79941777  2.48236495  2.42356419  0.30943042]
 [-0.70078618 -0.41511584 -0.16075614 -0.16047844 -0.25672933]
 [ 0.05149901 -0.00271409 -0.23091725 -0.23512403  2.1772287 ]
 [ 1.16037972 -0.37741377 -0.08700996 -0.09501774 -0.15844696]
 [-0.31405921  1.68670155 -0.57387501 -0.53664509 -0.17239155]]
[3 3 3 ... 3 3 3]
